In [1]:
path = "data/data186231"
! ls -l $path

总用量 1350
-rwxrwxrwx 1 aistudio aistudio  204231 1月  13 21:37 predict.csv
-rwxrwxrwx 1 aistudio aistudio   45018 1月  13 21:37 sample_submission.csv
-rwxrwxrwx 1 aistudio aistudio 1132280 1月  13 21:37 train.csv


In [16]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [3]:
train = pd.read_csv(f"{path}/train.csv")
train["time"] = pd.to_datetime(train["time"])
train

,time,dv1,dv2,mv1,cv1,cv2
0,2022-11-17 00:00:00,0.97,351.13,305.50,324.85,0.91
1,2022-11-17 00:00:30,1.05,349.27,310.28,325.30,0.87
2,2022-11-17 00:01:00,0.96,350.02,315.82,324.13,0.86
3,2022-11-17 00:01:30,1.03,351.71,318.22,327.08,0.90
4,2022-11-17 00:02:00,1.01,348.61,315.94,323.70,0.83
...,...,...,...,...,...,...
20005,2022-11-23 22:42:30,1.07,352.71,305.25,385.00,0.09
20006,2022-11-23 22:43:00,1.05,353.16,304.01,385.48,0.10
20007,2022-11-23 22:43:30,1.07,352.81,305.67,382.68,0.08
20008,2022-11-23 22:44:00,1.06,352.88,304.87,385.86,0.10


In [4]:
COLS = ["time1", "time2", "dv1", "dv2", "mv1", "cv1", "cv2"]

train1 = train.copy()
train1["time1"] = train1["time2"] = train1["time"]
train1 = train1[COLS]
train1

,time1,time2,dv1,dv2,mv1,cv1,cv2
0,2022-11-17 00:00:00,2022-11-17 00:00:00,0.97,351.13,305.50,324.85,0.91
1,2022-11-17 00:00:30,2022-11-17 00:00:30,1.05,349.27,310.28,325.30,0.87
2,2022-11-17 00:01:00,2022-11-17 00:01:00,0.96,350.02,315.82,324.13,0.86
3,2022-11-17 00:01:30,2022-11-17 00:01:30,1.03,351.71,318.22,327.08,0.90
4,2022-11-17 00:02:00,2022-11-17 00:02:00,1.01,348.61,315.94,323.70,0.83
...,...,...,...,...,...,...,...
20005,2022-11-23 22:42:30,2022-11-23 22:42:30,1.07,352.71,305.25,385.00,0.09
20006,2022-11-23 22:43:00,2022-11-23 22:43:00,1.05,353.16,304.01,385.48,0.10
20007,2022-11-23 22:43:30,2022-11-23 22:43:30,1.07,352.81,305.67,382.68,0.08
20008,2022-11-23 22:44:00,2022-11-23 22:44:00,1.06,352.88,304.87,385.86,0.10


In [5]:
train2 = [train1]

for n in tqdm(range(1, len(train)+1)):
    _train2 = train.head(n).copy()
    _train2["k"] = 1
    _train2 = _train2.groupby(by="k").agg({
        "time": ["max", "min"],
        "dv1": "mean", 
        "dv2": "mean", 
        "mv1": "mean", 
        "cv1": "mean", 
        "cv2": "mean",
    })
    _train2.columns = [
        f"{icol1}{icol2}".replace("mean", "").replace("max", "1").replace("min", "2")
        for icol1, icol2 in _train2.columns]
    train2.append(_train2)

train2 = pd.concat(train2)
train2

100%|██████████| 20010/20010 [03:49<00:00, 87.01it/s]


,time1,time2,dv1,dv2,mv1,cv1,cv2
0,2022-11-17 00:00:00,2022-11-17 00:00:00,0.970000,351.130000,305.500000,324.850000,0.910000
1,2022-11-17 00:00:30,2022-11-17 00:00:30,1.050000,349.270000,310.280000,325.300000,0.870000
2,2022-11-17 00:01:00,2022-11-17 00:01:00,0.960000,350.020000,315.820000,324.130000,0.860000
3,2022-11-17 00:01:30,2022-11-17 00:01:30,1.030000,351.710000,318.220000,327.080000,0.900000
4,2022-11-17 00:02:00,2022-11-17 00:02:00,1.010000,348.610000,315.940000,323.700000,0.830000
...,...,...,...,...,...,...,...
1,2022-11-23 22:42:30,2022-11-17 00:00:00,1.118441,351.827909,283.200823,349.133016,0.475813
1,2022-11-23 22:43:00,2022-11-17 00:00:00,1.118438,351.827975,283.201863,349.134833,0.475794
1,2022-11-23 22:43:30,2022-11-17 00:00:00,1.118435,351.828024,283.202986,349.136509,0.475774
1,2022-11-23 22:44:00,2022-11-17 00:00:00,1.118432,351.828077,283.204069,349.138345,0.475755


In [50]:
train2["itime1"] = train2["time1"].astype('int64')//1e9
train2["itime2"] = train2["time2"].astype('int64')//1e9

In [61]:
testa = pd.read_csv(f"{path}/predict.csv")
testa

,time,dv1,dv2,mv1,cv1,cv2
0,2022/11/23 23:59:30,1.00,350.00,300.00,324.0,0.877
1,2022/11/24 0:00:00,1.02,349.93,331.86,323.7,0.874
2,2022/11/24 0:00:30,1.02,349.89,331.34,NaN,NaN
3,2022/11/24 0:01:00,1.01,350.04,331.06,NaN,NaN
4,2022/11/24 0:01:30,1.01,349.84,331.39,NaN,NaN
...,...,...,...,...,...,...
4996,2022/11/25 17:37:30,0.60,349.45,302.84,NaN,NaN
4997,2022/11/25 17:38:00,0.61,349.06,302.76,NaN,NaN
4998,2022/11/25 17:38:30,0.57,349.98,303.15,NaN,NaN
4999,2022/11/25 17:39:00,0.61,350.29,302.45,NaN,NaN


In [69]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model

result = pd.DataFrame()
for T in ["cv1", "cv2"]:
    x, y = train2[["dv1", "dv2", "mv1"]], train2[T]

    def stdError_func(y_test, y):
        return np.sqrt(np.mean((y_test - y) ** 2))

    def R2_1_func(y_test, y):
        return 1 - ((y_test - y) ** 2).sum() / ((y.mean() - y) ** 2).sum()

    def R2_2_func(y_test, y):
        y_mean = np.array(y)
        y_mean[:] = y.mean()
        return 1 - stdError_func(y_test, y) / stdError_func(y_mean, y)


    D = 10
    poly_reg = PolynomialFeatures(degree=D-1)
    X_ploy = poly_reg.fit_transform(x)
    lin_reg_2 = linear_model.LinearRegression()
    lin_reg_2.fit(X_ploy,y)
    predict_y = lin_reg_2.predict(X_ploy)
    strError = stdError_func(predict_y, y)
    R2_1 = R2_1_func(predict_y, y)
    R2_2 = R2_2_func(predict_y, y)
    score = lin_reg_2.score(X_ploy, y)

    print(T)
    print("coefficients", lin_reg_2.coef_[:5])
    print("intercept", lin_reg_2.intercept_)
    print('degree={}: strError={:.2f}, R2_1={:.2f},  R2_2={:.2f}, clf.score={:.2f}'.format(D, strError, R2_1, R2_2, score))
    testa[f"_{T}"] = lin_reg_2.predict(poly_reg.transform(testa[["dv1", "dv2", "mv1"]]))

    result[T] = [_2 if pd.isna(_1) else _1 for _1, _2 in zip(testa[T], testa[f"_{T}"])]


cv1
coefficients [-2.17414781e-14 -8.98612593e-10 -1.93494889e-10 -1.77060436e-11
  4.44738788e-09]
intercept -53620.93346278137
degree=10: strError=12.36, R2_1=0.84,  R2_2=0.59, clf.score=0.84
cv2
coefficients [ 5.88205025e-16 -6.27255553e-12 -2.99768168e-13  1.33638127e-12
 -3.86371267e-11]
intercept 1536.0986297811994
degree=10: strError=0.15, R2_1=0.57,  R2_2=0.35, clf.score=0.57


In [71]:
result.to_csv("submit.csv", index=None)